# Modelo de Black-Litterman

El modelo permite, a partir del CAPM, incluir información cualitativa en la definición de las carteras. Se asume que la matriz de covarianza es dada y conocida, y nos centramos en las primas de riesgo. Lógicamente desde la perspectiva del CAPM el modelo es inutil, ya que si el mercado es eficiente no es posible batirle con información cualitativa.

In [10]:
import numpy as np
import pandas as pd

numpy trata un vector columna de manera diferente a un array unidimensional. Para usar vectores columna de manera consistente, la siguiente función `as_colvec` permite trabajar con un numpy o una matriz numpy de una columna (es decir, un vector columna) y devuelve los datos como un vector columna. 

In [11]:
def as_colvec(x):
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x, axis=1)

In [12]:
np.arange(4)

array([0, 1, 2, 3])

In [13]:
as_colvec(np.arange(4))

array([[0],
       [1],
       [2],
       [3]])

Estimar la rentabilidad implícitas en equilibrio suponiendo que tenemos dos activos Intel (INTC) y Pfizer (PFE) cuya matriz de varianza convarianza es (multiplicados por $10^4$):
$$\Sigma=\left[\begin{array}{crl}  
46.0 & 1.06 \\
1.06 & 5.3
\end{array}\right]$$

La capitalización de Intel es aproximadamente 80.000 millones de US$ y la de Pfizer de 100.000 millones US$. Si creamos una cartera de mercado basada en su capitalización obtenemos las siguientes ponderaciones:

$$ W_{INTC}=80/180=0.44, W_{PFE}=100/180=0.56 $$

Considerar que $\lambda=2.5$.


In [20]:
def implied_returns(landa, sigma, w):
    """
Obtenemos los rendimientos implícitos en equilibrio por ingeniería inversa.
Inputs:
landa: Coeficiente de Aversión al Riesgo (escalar)
sigma: Matriz de Varianzas-Covarianzas (N x N) como DataFrame
    w: Ponderaciones de la cartera de mercado (N x 1) como Series
Rentabilidades vector N x 1 como Series

`squeeze()`es una herramienta útil para manejar y simplificar 
las dimensiones de los arrays, pero no modifica los datos ni 
las operaciones matemáticas subyacentes como 
el producto de matrices
       """
    ir.name = 'Implied Returns'
    return ir


In [7]:
tickers = ['INTC', 'PFE']
s = pd.DataFrame([[46.0, 1.06], [1.06, 5.33]], index=tickers, columns=tickers) *  10E-4
pi = implied_returns(landa=2.5, sigma=s, w=pd.Series([.44, .56], index=tickers))
pi

INTC    0.052084
PFE     0.008628
Name: Implied Returns, dtype: float64

La opinión de los analistas disiente de las rentabilidades de equilibrio estimadas y considera que Intel reducirá su rentabilidad al 2% y Pfizer la incrementará al 4%. Por tanto vamos a construir una nueva cartera ya no en el modelo de mercado, sino a partir del Modelo de Markowitz. Por tanto, optimizaremos el ratio de Sharpe considerando que la rentabilidad libre de riesgo es del 0%. 
$$\begin{align}
\mu_e- R_f &=\Sigma z \\
z &=\Sigma^{-1} (\mu_e- R_f) \\
w_{SR_i} &=\frac{z_i}{\sum_{i=1}^{N}z_i} 

\end{align}$$

In [18]:
mean_returns_minus_rf=([.02, .04])
z = np.dot(np.linalg.inv(s), mean_returns_minus_rf)
z

array([0.26305395, 7.45237576])

In [19]:
w_sr = np.divide(z, np.sum(z))
w_sr

array([0.03409453, 0.96590547])

La optimización ingenua de Markowitz propone una cartera con una ponderación poco realista de más del 96% en Pfizer y menos del 4% en Intel. Esto es completamente impracticable y ningún inversor razonable haría apuestas tan dramáticas.

## La propuesta del modelo de Black Litterman

Black Litterman propone una alternativa para considera $k$ opiniones sobre la rentabilidad de los activos. Partiendo de la estadística Bayesiana consideran que la rentabilidad de los activos es una variable aleatoria inobservable y que únicamente podremos inferir su función de distribución. La inferencia parte de una creencia a priori. Se utiliza la información adicional para inferior la función de distribución a posteriori. En dicho marco la cartera de equilibrio propuesta por el CAPM es la creencia a priori y la información generada son las opiniones. La estadística bayesiana nos permitirá determinar la función de distribución a posteriori.

La rentabilidad de los $N$ activos que conforman el mercado es una variable aleatoria:
$$r\sim N(\mu,\Sigma) \tag{1}$$

La catera de equilibrio viene determinada por $w_eq$. Si  $\delta$ representa la aversión al riesgo, La prima por riesgo en equilibrio será (Black 1989):
$$\Pi=\delta\Sigma^{-1}w_eq \tag{2}$$

Desde la perspectiva Bayesiana la función a priori de la rentabilidad esperada, está centrada en el valor de equilibrio, es decir, se comporta como un normal de media $\Pi$
$$\mu=\Pi+\varepsilon^{(e)} \tag{3}$$

donde $\varepsilon^{(e)}$ es un vector aleatorio que se distribuye como una normal de media cero y con una matriz de covarianza $\tau\Sigma$, donde $\tau$ es un indicador de la incertidumbre del CAPM a priori.

Se disponen de $K$ opiniones que se comportan como una normal de media $q$ y desviación típica $\omega$. Dichas opiniones se representan con dos vectores $P$ y $Q$:
$$P'=(p_1, p_2, ...p_k)\tag{4}$$
$$Q'=(q_1, q_2, ...q_k)\tag{5}$$

La opinión de los inversores puede expresar como:

$$P\mu=Q+\varepsilon^{(\nu)} \tag{6}$$

donde $\varepsilon^{(\nu)}$ es una variable aleatoria inobservable normalmente distribuida con media cero y matriz de covarianza $\Omega$. Se supone que las opiniones son únicas e incorrelacionadas con el resto. Ello permite que la matriz de covarianza que refleja la incertidumbre implícita en las opiniones $\Omega$ sea una matriz diagonal que recoja la varianza de las opiniones $(\omega_i)$. Además se asume que $\varepsilon^{(e)}$ son independientes $\varepsilon^{(\nu)}$:

$$\left(\begin{array}{crl}  
\varepsilon^{(e)} \\
\varepsilon^{(\nu)}
\end{array}\right) \sim 
N\left(0,\left[\begin{array}{crl}  
\tau\Sigma & 0 \\
0 & \Omega
\end{array}\right]\right)
\tag{7}$$

Las opiniones se combinan con el CAPM en el marco de la estadística Bayesiana y el restulatod es que las rentabilidades esperadas se dsitribuyen como $N(\bar{\mu},\bar{M}^{-1} )$. Donde la rentabilidad esperada es:

$$\bar{\mu}=\left[(\tau\Sigma)^{-1}+P'\Omega^{-1}P\right]^{-1}\left[(\tau\Sigma)^{-1}\Pi+P'\Omega^{-1}Q\right] \tag{8}$$

y la matriz de covarianza $\bar{M}^{-1}$ es dado por:

$$\bar{M}^{-1}=\left[(\tau\Sigma)^{-1}+P'\Omega^{-1}P\right]^{-1} \tag{9}$$

In [31]:
# Opinión absoluta 1: la rentabilidad de INTC será del 2%
# Opinión absoluta 2: la rentabilidad de PFE será del 4%
q = pd.Series({'INTC': 0.02, 'PFE': 0.04})

# La Matriz P (Pick) 
p = pd.DataFrame([
# For View 1, this is for INTC
    {'INTC': 1, 'PFE': 0},
# For View 2, it is for PFE
    {'INTC': 0, 'PFE': 1}
    ])

q

INTC    0.02
PFE     0.04
dtype: float64

In [33]:
# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

In [35]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

In [36]:
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_prior=pd.Series({'INTC':.44, 'PFE':.56}), sigma_prior=s, p=p, q=q)
# Black Litterman Implied Mu
bl_mu


NameError: name 'ir' is not defined

# Modelo de Black-Litterman

El modelo permite, a partir del CAPM, incluir información cualitativa en la definición de las carteras. Se asume que la matriz de covarianza es dada y conocida, y nos centramos en las primas de riesgo. Lógicamente desde la perspectiva del CAPM el modelo es inutil, ya que si el mercado es eficiente no es posible batirle con información cualitativa. 

Partimos de la función de utilidad: 

$$U=w^T\Pi -\frac{\lambda}{2}w^T\Sigma w \tag{1}$$

- $U$ es la función de utilidad que deseamos optimizar de acuerdo al CAPM
- $w$ vector de ponderaciones
- $\Pi$ vector de premio de mercado o exceso de rentabilidad 
- $\lambda$ aversión al riesgo. Distintos autores utilizan una lambda arbitraria próxima a 2. Una aproximación alternativa es que $\lambda=\frac{\mu_M}{\sigma^2_M}$ siendo $\mu_M$ y $\sigma^2_M$ la rentabilidad media y la varianza de la cartera de mercado. Muchas veces se utiliza un indice como aproximación a la cartera de mercado.
- $\Sigma$ matriz de covarianza

La función objetivo que utilizada para la identificación de las carteras eficientes:

$$\underset{w}{max}w^T \Pi-\frac{\lambda}{2} w^T \Sigma w \tag{2}$$

Dado que $U$ es convexa aplicamos la primera condición de optimización:

$$\frac{dU}{dw}=\Pi -\lambda\Sigma w=0 \tag{3}$$

La primas por riesgo en equilibrio vienen dado por:

$$\Pi=\lambda\Sigma w \tag{4}$$

Con el objeto de estimar la cartera de mercado necesito el valor de la aversión al riesgo del mercado $\lambda$. Multiplico ambos lados de la ecuación por $w^T$:

$$(r_M-r_f)=\lambda \sigma^2 \tag{5}$$

Definimos la formula 5 en función del ratio de Sharpe (SR):

$$\lambda=\frac{SR}{\sigma_M} \tag{6}$$

Las ponderaciones de la cartera de mercado (M) serán:

$$w_M=\frac{1}{\lambda}\Sigma^{-1}\Pi \tag{7}$$


### Estimación de $\Pi_{eq}$

A partir de un ejemplo estimaremos, a partir de la ecuación 4 los rendimientos de los activos:  $$\Pi_{eq}=\lambda\Sigma w $$





$$P(B|A)\sim N\left(P^{-1}Q, \left[P^T\Omega^{-1} P \right]^{-1} \right) \tag{11}$$

donde $k$ es el número de opiniones subjetivas:

- $P\in \mathbb{R}^{k\times n}$ es la matriz de ponderaciones de los activos. En la literatura se utilizan distintas ponderaciones, así, He y Litterman (1999) e Idzorek (2005) utilizan un esquema ponderado por la capitalización del mercado, mientras que Satchell y Scowcroft (2000) utilizan un esquema equiponderado. En la práctica, las ponderaciones serán una mezcla que dependerá del proceso utilizado para estimar los rendimientos.  
- $Q\in \mathbb{R}^k$ es el vector de rendimietnos tras incorporar las opiniones.
- $\Omega \in \mathbb{R}^{k\times k}$ es la matriz de covarianzas de las opiniones, como ya hemos indicado es una matriz diagnoal. $\Omega^{-1}$ es conocido como la confianza en la visión de los inversores.


Black-Litterman proponen una simplificación en su modelo, la matriz de covarianzas utilizada  $(\Sigma_\Pi)$ es proporcional a la matriz de covarianzas histórica $\Sigma$. Para ello, proponen un parámetro $\tau$ que es una constante que indica la incertidumbre a prioir: $\Sigma_\Pi=\tau\Sigma$. Dicho parámetro es fuente de confusión, algunos los igualan a 1 o calibran el modelo con $\tau$. Black y Litterman proponen un número bajo, en este sentido una técnica habitual es $\tau=1/T$ donde T es el número de periodos utilizados.

El modelo de Black-Litterman utiliza la estadística bayesiana para integrar dos funciones de distribución, una vinculada a la información histórica y otra relacionada con las opiniones subjetivas. Partimos de la ley de Bayes propone:

$$P(A|B)=\frac{P(B|A)P(A)}{P(B)}\tag{8}$$

donde $P(A|B)$ es la probabilidad de A condicionada a que ocurra B, $P(B|A)$ es la probabiliad de B condicionada a que ocurra A, $P(A)$ y $P(B)$ son las probabilidades no condicionadas. En términos generales $P(A)$ se denomina distribución a priori y $P(B|A)$ es la probabilidad condicional de los datos observados a priori, es la probabilidad a posteriori.

El primer componente, la distribución a priori, se define de la siguiente manera:

$$ P(A)\sim N(\pi_e,\tau \Sigma), r_A\sim N(P(A), \Sigma)\tag{9} $$

Estamos indicando que la probabilidad a priori se distribuye como una normal cuya esperanza es la prima por riesgo esperada en equilibrio y su varianza es proporcional a la matriz de covarianzas histórica. 

Podemos redefinir la fórmula 8:

$$ r_A\sim N(\pi_e,(1+\tau) \Sigma) \tag{10} $$

La rentabilidad estimada $r_A$ es la resultante de crear una cartera que invierte $1/(1+\tau)$ en la cartera de mercado y $\tau/(1+\tau)$ en el activo libre de riesgo:

$$\begin{aligned}
\Pi&=\lambda\Sigma w \\
w&=(\lambda\Sigma)^{-1} \Pi \\
\hat{w}&=((1+\tau)\lambda\Sigma)^{-1} \Pi \\
\hat{w}&=(1/1+\tau)(\lambda\Sigma)^{-1} \Pi \\
\hat{w}&=(1/1+\tau)w
\end{aligned}$$

En la Figura 1 se representa tando la cartera de mercado derivada del CAPM como la cartera propuesta por Black-Litterman, sin incluir las opiniones.

Figura 1: Comparación entre la cartera de mercado propuesta por el CAPM y la cartera de Black-Litterman sin tener en cuenta las opiniones.

![BL_CAPM](https://raw.githubusercontent.com/alfonso-santos/Python-aplicado-gestion-carteras/fernando/Tema_2/imgs/BL_Capm.jpg)

Fuente:Walters (2005)

El segundo componente, la probabilidad condicional, son las opiniones que las consideramos aleatorias y le imponemos las siguientes condiciones:
1. Se requiere que las opiniones sean únicas e incorrelacionadas con el resto. Ello permite que la matriz de covarianza que refleja la incertidumbre implícita en las opniones $\Omega$ sea una matriz diagonal que recoja la varianza de las opiones.
2. Se invierte la totalidad del capital.

$$P(B|A)\sim N\left(P^{-1}Q, \left[P^T\Omega^{-1} P \right]^{-1} \right) \tag{11}$$

donde $k$ es el número de opiniones subjetivas:

- $P\in \mathbb{R}^{k\times n}$ es la matriz de ponderaciones de los activos. En la literatura se utilizan distintas ponderaciones, así, He y Litterman (1999) e Idzorek (2005) utilizan un esquema ponderado por la capitalización del mercado, mientras que Satchell y Scowcroft (2000) utilizan un esquema equiponderado. En la práctica, las ponderaciones serán una mezcla que dependerá del proceso utilizado para estimar los rendimientos.  
- $Q\in \mathbb{R}^k$ es el vector de rendimietnos tras incorporar las opiniones.
- $\Omega \in \mathbb{R}^{k\times k}$ es la matriz de covarianzas de las opiniones, como ya hemos indicado es una matriz diagnoal. $\Omega^{-1}$ es conocido como la confianza en la visión de los inversores.

Un ejemplo de construcción de las matrices de opiniones puede ser el siguiente. Disponemos de cuatro activos y dos opiniones. El activo 1 será más productivo que el activo 3 en un 2% con una confinaza $\omega_1$. El activo 2 tendrá una rentabilidad del 3% con un nivel de confianza $\omega_2$. La especificación de las opiniones:

$$P=\left[\begin{array}{crl}  
1 &0 &-1 &0 \\
0 &1 &0  &0
\end{array}\right]; 
Q=\left[\begin{array}{crl}  
0.02  \\
0.03 
\end{array}\right];
\Omega=\left[\begin{array}{crl}  
\omega_1 &0  \\
0 & \omega_2
\end{array}\right]$$

## La especificación de $\Omega$

Dado que $\left[P^T\Omega^{-1} P \right]$ probablemente no sea invertible y por tanto no se podrá utilizar. Afortunadamente disponemos de distintos métodos para especificar $\Omega$:
- Utilizar una proporción de la varianza de la opinión.
- Utilizar un intervalo de confianza.
- Utilizar la varianza de los residuos en un modelo factorial.
- Utilizar el método de Idzorek para especificar la confianza a lo largo de la dimensión de las ponderaciones.

### Utilizar una proporción de la varianza de la opinión

Se asume que la varianza de las opiones son proporcionales a la varianza de la rentabilidad de los activos (He and Litterman. 1999; Meucci, 2006):

$$\begin{aligned}
\omega_{ij}&=p(\tau\Sigma)p^T \forall i=j \tag{12}\\
\omega_{ij}&=0 \\
\Omega&=diag\left(P (\tau\Sigma) P^T \right)
\end{aligned}$$

### Utilizar un intervalo de confianza.

El inversor especifica la varianza utilizando un intervalo de confianza en torno a la rentabilidad media. Por ejemplo, se ha estimado una rentabilidad esperada del 3% y se le asigna una probabilidad del 68% de se encuentre en el intervalo (2%,4%), por tanto la desviación típica es del 1%.

###  Utilizar la varianza de los residuos en un modelo factorial

La predicción de la rentabilidad puede estar basada en un modelo factorial.

$$r=\sum_{i=1}^{n} \beta_i f_i+ \varepsilon \tag{13}$$

donde:
- $r$ es la rentabilidad del activo.
- $\beta_i$ es el coeficiente del factor $i$.
- $f_i$ es la rentabilidad del factor $i$.
- $\varepsilon$ es el error que presenta una distribución normal. 

La varianza de la rentabilidad estimada a través del modelo factorial es:

$$V(r)=BV(F)B^T+V(\varepsilon) \tag{14}$$

donde:
- $B$ es el vector de coeficientes
- $F$ es el vector de rentabilidad de los factores

Beach y Orlov (2006) realizan las predicciones utilizando un modelo factorial GARCH que posteriormente incluye en el modelo de  Black-Litterman. 

### Utilizar el método de Idzorek para especificar la confianza a lo largo de la dimensión de las ponderaciones

Idzorek (2005) describe un método para especificar la confianza en la vista en términos de un desplazamiento porcentual de los pesos en el intervalo desde el 0% de confianza hasta el 100% de confianza. Veremos el algoritmo de Idzorek en la sección sobre extensiones

## La estimación del modelo

Empezaremos exponiendo el modelo de estimación mixta de Theil pues es más simple que el modelo Bayesiano.

### Modelo de estimación mixta de Theil

El modelo se inicia con un modelo lineal:

$$x\beta=\pi+u \tag{15}$$

donde:

- $\pi$ es el vector $n\times 1$ de primas pro riesgo en equilibrio
- $x$ es una matriz $n\times n$ que representa los factores del modelo
- $\beta$ es un vector $n\times 1$ que representea los coeficientes del modelo
- $u$ es un vector $n\times 1$ que representa los residuos del modelo $E(u)=0; V(u)=E(u'u)=\Phi$. Siendo $\Phi$ nos singular.

El modelo de Black-Litterman utiliza un modelo lineal muy simple, la rentabilidad de cada título es modelizada con una variable dummy. Por tanto, $x$, es la matriz identidad. Dado que $\beta$ y $u$ son independientes y $x$ es constante, la varianza de la prima por riesgo:

$$V(\pi)=x'V(\beta)+V(u) \tag{16}$$

Lo cual puede simplificarse

$$V(\pi)=\Sigma+\Phi \tag{17}$$

donde: 
- $\Sigma$ es la matriz de covarianza histórica
- $\Phi$ es la matriz de covarianza de los residuos.

Ahora vamos a considerar información adicional que combinaremos con la histórica. Esta información puede ser cualitativa o derivada de modelos estadísticos. Además no tiene porque estimarse para todos los activos:

$$p\Beta=q+\nu \tag{18}$$

donde:
- $q$ es un vector $k\times 1$ que recoge las opiniones sobre las rentabilidades
- $p$ es una matriz $k \times n$ que permite modeliza las opiniones
- $\beta$ es un vector $n\times 1$ que recoge los coeficientes 
- $\nu$ es un un vector $k\times 1$ donde $E(\nu)=0, V(\nu)=E(\nu'\nu)=\Omega$ y $\Omega$ es no singular.

Vamos a combinar la información a priori y la información condicional:
$$\left[\begin{array}{crl}  
x \\
p
\end{array}\right] \hat{\beta}=\left[\begin{array}{crl}  
\pi\\
q
\end{array}\right]+\left[\begin{array}{crl}  
u \\
\nu
\end{array}\right] \tag{19}$$

Donde el valor esperado de los residuos es cero y la varianza de los residuos es
$$V\left(\left[\begin{array}{crl}  
u \\
\nu
\end{array}\right]\right)= E\left(\left[\begin{array}{crl}  
u \\
\nu
\end{array}\right]\left[\begin{array}{crl}  
u' \nu'
\end{array}\right]\right)= \left[\begin{array}{crl}  
\phi &0\\
0 &\nu
\end{array}\right]\tag{20}$$

Por minimos cuadrados ordinarios podemos estimar las $\hat{\beta}$:

$$\hat{\beta}=\left[\left[\begin{array}{crl}  
x' p'
\end{array}\right] \left[\begin{array}{crl}  
\phi &0\\
0 &\nu
\end{array}\right]^{-1} \left[\begin{array}{crl}  
x \\
p
\end{array}\right]\right]^{-1}\left[\begin{array}{crl}  
x' p'
\end{array}\right] \left[\begin{array}{crl}  
\phi &0\\
0 &\nu
\end{array}\right]^{-1} \left[\begin{array}{crl}  
\pi \\
q
\end{array}\right]\tag{21}$$

Podríamos expresar así la ecuación 21:

$$\hat{\beta}=\left[x'\Phi^{-1}x+p'\Omega^{-1}p\right]^{-1}\left[x'\Phi^{-1}\pi+p'\Omega^{-1}q\right]^{-1}$$


La distribución posterior, es la distribución de las rentabilidades considerando las rentabilidad de equilibrio y las opiniones. Se estimará como el producto de las dos distribuciones previas y se podrá expresar así:
$$\begin{aligned}
P(A|B) &\sim N(\mathbb{E}[r], var(r)) \tag{17}\\
(\mathbb{E}[r]=
\end{aligned}$$


Definimos $w_{Mj}$ como la ponderación del activo $j$ en la cartera de mercado $M$. Por tanto la rentabilidad de la cartera de mercado viene definida por:

$$\tilde{r}_M=\sum_{j=1}^n w_{Mj}\tilde{r}_j \tag{3}$$

De acuerdo al CAPM:

$$\mu_j-r_f=\beta_j(\mu_M-r_f) \tag{4}$$

donde

$$\mu_M=\sum_{j=1}^n w_{Mj}\mu_j \tag{5}$$

El premio por el riesgo será:

$$\pi_{ej}=\mu_j-r_f \tag{6}$$

el subíndice $e$ se utiliza para enfatizar que se trata de un mercado en equilibrio. Si desarrollamos la ecuación anterior:

$$\begin{gather}
\pi_{ej}=\frac{Cov(\tilde{r}_i,\tilde{r}_M)}{\sigma_M^2}(\mu_M-r_f) \tag{7} \\
=\frac{1}{\sigma_M^2}Cov\left(\tilde{r}_i,\sum_{j=1}^{n} w_{Mj}\tilde{r}_j \right)(\mu_M-r_f) \\ 
=\frac{\mu_M-r_f}{\sigma_M^2} \sum_{j=1}^{n} w_{Mj} Cov(\tilde{r}_i,\tilde{r}_j ) 
\end{gather}$$






Podemos expresar matricialmente la ecuación 7:

$$\pi_e=\delta\Sigma w_M \tag{8}$$

y definimos el precio de mercado del riesgo

$$\delta=\frac{\mu_M-r_f}{\sigma_M^2} \tag{9}$$

Si aplicamos la estadística Bayesiana la prima por riesgo $pi_e$ es considerada como un vector de variables aleatorias $\tilde{\theta}$ condicionada por la información disponible $\mathcal{M}_e$, por tanto:

$$\pi_e=E\left[\tilde{\theta}\left. \right|\mathcal{M}_e\right] \tag{10}$$

La notación $\pi_S$ representa la prima por riesgo subjetiva y $\pi_{BL}$ es la prima por riesgo propuesta por Black-Literrman. Suponemos que la rentabilidad y los parámetros se distribuyen como normales.

Tras un profundo análisis de la realidad empresarial el análisis propone que el activo 2 batirá la rentabilidad del activo 5 en un 2%, es decir:
$$\pi_{s2}-\pi_{s5}=0.02 \tag{11}$$

La propuesta subjetiva sobre la prima por riesgo puede expresarse como una relación lineal, desde una perspectiva matricial sería:

$$P\pi_{s}=Q \tag{12}$$





Lógicamente las opiniones subjetivas son aleatorias lo que exige expresar el nivel de confianza:

$$P\tilde{\theta} \sim N(Q, \Omega) \tag{13}$$

El vector de incertidumbre $\tilde{\theta}$ se comporta como una normal con valor esperado $q$ y matriz de covarianza $\Omega$, una matriz diagonal que incluye la varianza de cada opinión. Ello nos permite incluir el nivel de confianza en las opiniones subjetivas. 

El modelo de Black-Litterman nos permite integrar, a través de la estadística bayesiana, dos funciones de distribución, una vinculada la información histórica y otra relacionada con la opiniones subjetivas. Así, la ley de Bayes propone:

$$P(A|B)=\frac{P(B|A)P(A)}{P(B)}\tag{14}$$

donde $P(A|B)$ es la probabilidad de A condicionada a que ocurra B, $P(B|A)$ es la probabiliad de B condicionada a que ocurra A, $P(A)$ y $P(B)$ son las probabilidades no condicionadas. En términos generales $P(A)$ se denomina distribución a priori y $P(B|A)$ es la probabilidad condicional de los datos observados a priori. 

Así, consideramos que la probabilidad a priori es definida como la rentabilidad en equilibrio $P(A)$ y las opiniones como la probabilidad condicional a priori $P(B|A)$. $P(B)$ es una constante de normalización. Por último, $P(A|B)$ es la distribución posterior que buscamos.




